<a href="https://colab.research.google.com/github/Rumeysakeskin/NLP-Onnx-TensorRT/blob/main/intent_slot_distelbert_inference_tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda
!pip install nvidia-pyindex
!pip install nvidia-tensorrt
!pip install transformers
!pip install omegaconf
!pip install ruamel-yaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661975 sha256=b58a8f74cb559964dd89f5a300dbc6a8920e1bf37bc73345f2be70083656ea34
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 36.0 MB/s eta 0:00:00


In [ ]:
from jetson_voice_utils.trt_model import TRTModel
from transformers import AutoTokenizer
from omegaconf import OmegaConf

# from ruamel.yaml import YAML
# from omegaconf import DictConfig
import time
import numpy as np

class ENNemoDialogueInferencerTRT(object):
    def __init__(self):

        config_file = "confing/intent_slot_jetson.yaml"

        self.en_config = OmegaConf.load(config_file)

        self.en_config.model_path = "ENGLISH_nemo_model_DISTEL_BERT.onnx" #ENGLISH

        self.intent_slot_label_path = "/nemo_format/" #ENGLISH

        self.nlp_dynamic_shapes = False
        dynamic_shapes = {'max' : (1, self.en_config.model['language_model']['max_seq_length'])}  # (batch_size, sequence)
        if self.nlp_dynamic_shapes:
            dynamic_shapes['min'] = (1, 1)


        self.en_TRT_model = TRTModel(self.en_config, dynamic_shapes)

        # create tokenizer
        self.en_tokenizer = AutoTokenizer.from_pretrained(self.en_config.model['tokenizer']['tokenizer_name'])
        #self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        #self.tokenizer.add_special_tokens({'sep_token': '[SEP]'})

        self.null_slot = self.slot_labels()[-1]  # 'O' in assistant dataset - always the last label?




    def to_numpy(self, tensor):
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

    def normalize_logits(self, logits):
        """
        Normalize logits such that they are distributed between [0,1]
        """
        return np.exp(logits - np.log(np.sum(np.exp(logits), axis=-1, keepdims=True)))

    def intent_labels(self):
        """
        List of the intent class labels.
        """
        # Define intent label map
        intent_labels = []
        with open(self.intent_slot_label_path + 'dict.intents.csv', 'r') as file:
            for intent in file:
                intent_labels.append(intent.strip())

        return intent_labels

    def intent_label(self, index):
        """
        Return an intent label by index (with bounds checking)
        """
        return self.intent_labels()[int(index)] if index < len(self.intent_labels()) else 'Unknown_Intent'

    def slot_labels(self):
        """
        List of the slot class labels.
        """
        # Define slot label map
        slot_labels = []
        with open(self.intent_slot_label_path + 'dict.slots.csv', 'r') as file:
            for slot in file:
                slot_labels.append(slot.strip())

        return slot_labels

    def slot_label(self, index):
        """
        Return a slot label by index (with bounds checking)
        """
        return self.slot_labels()[int(index)] if index < len(self.slot_labels()) else self.null_slot

    def find_subtokens(self, encodings, method='char_span'):
        """
        Compute the subtoken mask, where each token is marked as True if it's a subtoken or False otherwise.
        Longer words/acronyms may be tokenized into mulitple word pieces (called subtokens), for example:

            'Yosemite' -> ['yo', '##se', '##mite']
            'U.S.' -> ['u', '.', 's', '.']

        Parameters:
          encodings (BatchEncoding) -- Output from tokenizer

          method (string) -- If 'char_span', the subtoken mask will be determined by looking at the character
                             indices.  Tokens that map to characters that are side-by-side are flagged as subtokens.

                             If 'subtoken_delimiters', subtokens will be identified by looking for '##' symbols.
                             However this can miss punctuated subtokens, such as 'U.S.'

        Returns boolean subtoken mask array with shape (num_queries, num_tokens)
        """
        num_queries = encodings['input_ids'].shape[0]
        subtoken_mask = []

        if method == 'char_span':
            for query_idx in range(num_queries):
                mask = []
                last_char = -1
                tokens = encodings.tokens(query_idx)

                for token_idx, word_id in enumerate(encodings.word_ids(query_idx)):
                    if word_id is None:  # skip special tokens
                        mask.append(False)
                        continue

                    chars = encodings.token_to_chars(query_idx, token_idx)

                    if chars[0] == last_char:
                        mask.append(True)
                    else:
                        mask.append(False)

                    last_char = chars[1]

                subtoken_mask.append(mask)

        elif method == 'subtoken_delimiters':
            for query_idx in range(num_queries):
                subtoken_mask.append([token.startswith('##') for token in encodings.tokens(query_idx)])
        else:
            raise ValueError(f"invalid method ('{method}')")

        return np.asarray(subtoken_mask)


    def inference(self, queries):

        start = time.time()
        lang = "en"

        encodings = getattr(self, f"{lang}_tokenizer")(
            text=query,
            padding='longest' if self.nlp_dynamic_shapes else 'max_length',
            truncation=True,
            max_length=getattr(self, f"{lang}_config").model['language_model']['max_seq_length'],
            return_tensors='np',
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            return_special_tokens_mask=True,
        )

        # during slot classification, we want to ignore slots from subtokens and special tokens
        subtoken_mask = self.find_subtokens(encodings, method='subtoken_delimiters')
        ignore_mask = subtoken_mask | encodings['special_tokens_mask']

        inputs = {}

        for input in getattr(self, f"{lang}_TRT_model").inputs:
            if input.name not in encodings:
                raise ValueError(f"the encoded inputs from the tokenizer doesn't contain '{input.name}'")

            inputs[input.name] = encodings[input.name]

        # run the model
        intent_logits, slot_logits = getattr(self, f"{lang}_TRT_model").execute(inputs)


        intent_logits = self.normalize_logits(intent_logits)
        slot_logits = self.normalize_logits(slot_logits)

        intent_preds = np.argmax(intent_logits, axis=-1)
        slot_preds = np.argmax(slot_logits, axis=-1)


        # convert numerical outputs to intent/slot labels
        results = []

        for query_idx, intent_id in enumerate(intent_preds):
            results.append({
                'intent' : self.intent_label(intent_id),
                'score' : intent_logits[query_idx][intent_id],
                #'slots' : []
            })

        for query_idx, slots in enumerate(slot_preds):
            query_slots = [self.slot_label(slot) for slot in slots]


            for token_idx, slot in enumerate(query_slots):
                # ignore unclassified slots or masked tokens
                if slot == self.null_slot or ignore_mask[query_idx][token_idx]:
                    continue

                # convert from token index back to the query string
                chars = encodings.token_to_chars(query_idx, token_idx)

                text = query[chars[0]:chars[1]]      # queries[query_idx]

                # append subtokens from the query to the text
                for subtoken_idx in range(token_idx+1, len(query_slots)):
                    if subtoken_mask[query_idx][subtoken_idx]:
                        subtoken_chars = encodings.token_to_chars(query_idx, subtoken_idx)

                        text += query[subtoken_chars[0]:subtoken_chars[1]]
                    else:
                        break

                results.append({
                    'slot' : slot,
                    'text' : text,
                    'score' : slot_logits[query_idx][token_idx][slots[token_idx]]
                })

        intent = ''
        intent_score = 0.0
        merged_data = {}

        for item in results:
            if 'intent' in item:
                intent = item['intent']
                intent_score = item['score']
            elif 'slot' in item:
                slot = item['slot']
                text = item['text']
                score = item['score']

                if slot not in merged_data:
                    merged_data[slot] = {'text': [], 'score': 0.0}

                merged_data[slot]['text'].append(text)

                if score > merged_data[slot]['score']:
                    merged_data[slot]['score'] = score

        return intent, intent_score, merged_data, time.time()-start

en_inference_model = ENNemoDialogueInferencerTRT()



binding 0 - 'input_ids'
   input:    True
   shape:    (1, 50)
   dtype:    DataType.INT32
   size:     200
   dynamic:  True
   profiles: [{'min': (1, 50), 'opt': (1, 50), 'max': (1, 50)}]


binding 1 - 'attention_mask'
   input:    True
   shape:    (1, 50)
   dtype:    DataType.INT32
   size:     200
   dynamic:  True
   profiles: [{'min': (1, 50), 'opt': (1, 50), 'max': (1, 50)}]


binding 2 - 'intent_logits'
   input:    False
   shape:    (1, 12)
   dtype:    DataType.FLOAT
   size:     48
   dynamic:  True
   profiles: []


binding 3 - 'slot_logits'
   input:    False
   shape:    (1, 50, 11)
   dtype:    DataType.FLOAT
   size:     2200
   dynamic:  True
   profiles: []



In [ ]:
query = "please prepare two sweet filter coffee with milk"

intent, intent_score, merged_data, inference_time = en_inference_model.inference(query)

print(f"\nQuery: \033[1m{query}\033[0m\n")

print(f"Predicted Intent: \033[1m{intent} (Score: {intent_score:.4f}\033[0m)\n")

print("Predicted Slot(s):")
for slot, values in merged_data.items():
    print(f"\033[1m{slot} --> {' '.join(values['text'])} (Score: {values['score']:.4f}\033[0m)")

print(f"\nInference time: \033[1m{inference_time:.4f} sec\033[0m")

inputs: {'input_ids': array([[  101,  7374,  2048,  4086, 11307,  4157,  2007,  6501,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])}
self.inputs: [<jetson_voice_utils.trt_binding.Binding object at 0x7f190e71f700>, <jetson_voice_utils.trt_binding.Binding object at 0x7f18769d3820>]

Query: please prepare two sweet filter coffee with milk

Predicted Intent: coffee_machine_brewing_query (Score: 0.9904)

Predicted Slot(s):
amount_per_person --> two (Score: 0.6788)
content --> sweet with milk (Score: 0.9168)
coffee_